In [240]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [241]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

from proyeccion_rdr.produccion.a05_unidades_de_apoyo import guardar_dict_en_excel

ANIOS_POBLACION = [str(i) for i in range(2017, 2036)]

In [242]:
# Lee la poblacion FONASA
ruta_poblaciones = "../data/interim/0.0_poblaciones_ine_y_fonasa_a_utilizar.xlsx"
poblaciones_fonasa = pd.read_excel(ruta_poblaciones, sheet_name="poblacion_FONASA")

# Filtra solamente la poblacion del SSMN y entre 0 y 15 anios (Poblacion que efectivamente
# atendieron). Elimina la columna del 2023, ya que esta mal calculada
poblaciones_fonasa_ssmn = (
    poblaciones_fonasa.query("Estrato == 'SSMN' and `Edad Incidencia` == 'entre_0_y_15'")
    .set_index(["Edad Incidencia", "Estrato"])
    .drop(columns=["2023"])
)
display(poblaciones_fonasa_ssmn)

,,2018,2019,2020,2021,2022
Edad Incidencia,Estrato,,,,,
entre_0_y_15,SSMN,148276,155320,153830,151308,153566


In [243]:
# Lee la poblacion SSMN extrapolada
estratos_interes_0_a_14 = ["recien_nacidos_vivos", "entre_1_y_14"]
estratos_interes_15_a_18 = ["entre_15_y_18"]
estratos_interes_0_a_18 = ["recien_nacidos_vivos", "entre_1_y_14", "entre_15_y_18"]
poblacion_fonasa_extrapolada_ssmn = pd.read_excel(
    ruta_poblaciones, sheet_name="poblaciones_fonasa_extrapoladas"
).query("Estrato == 'SSMN'")

# Extrae solo la poblacion entre 0 y 14 anios
poblacion_fonasa_extrapolada_ssmn_0_a_14 = poblacion_fonasa_extrapolada_ssmn.query(
    "`Edad Incidencia`.isin(@estratos_interes_0_a_14)"
)[ANIOS_POBLACION].sum()


# Extrae solo la poblacion entre 15 y 18 anios
poblacion_fonasa_extrapolada_ssmn_15_a_18 = poblacion_fonasa_extrapolada_ssmn.query(
    "`Edad Incidencia`.isin(@estratos_interes_15_a_18)"
)[ANIOS_POBLACION].sum()

# Extrae la poblacion total
poblacion_fonasa_extrapolada_ssmn_0_a_18 = poblacion_fonasa_extrapolada_ssmn.query(
    "`Edad Incidencia`.isin(@estratos_interes_0_a_18)"
)


In [244]:
# Lee las consultas de urgencias REM
produccion_urgencias_rem = pd.DataFrame(
    [
        [59, 70, 82, 65, 87, 69, 91, 448, 176, 169],
        [1455, 1292, 2133, 2635, 3096, 1499, 5235, 13613, 12473, 12981],
        [33159, 28148, 25438, 22907, 27428, 13795, 20488, 29809, 28980, 29070],
        [66592, 64954, 63160, 62138, 54425, 21814, 22329, 36763, 37584, 38696],
        [4841, 4781, 0, 0, 1, 0, 0, 0, 0, 0],
        [3764, 3302, 5514, 11914, 8251, 2476, 1484, 933, 1237, 1250],
        [109870, 102547, 96327, 99659, 93288, 39653, 49627, 81566, 80450, 82166],
    ],
    index=["C1", "C2", "C3", "C4", "SIN CATEGORIZACION", "C5", "Total"],
    columns=["2015", "2016", "2017", "2018", "2019", "2020", "2021", "2022", "2023", "2024"],
)
total_urgencias_rem = produccion_urgencias_rem.loc["Total"]

# Obtiene el porcentaje de cada categoria respecto al total
porcentaje_categorizacion_rem = produccion_urgencias_rem / produccion_urgencias_rem.iloc[-1]

In [245]:
# Obtiene la proporcion de urgencias y poblacion
proporcion_urgencias_y_poblacion = total_urgencias_rem / poblaciones_fonasa_ssmn

In [246]:
# Selecciona la cantidad de urgenicas por personas.
# De 15 a 18 fue criterio de experto - Dr Tapia
anio_maximo = proporcion_urgencias_y_poblacion.idxmax(axis=1).iloc[0]
factor_urgencias_0_a_14 = proporcion_urgencias_y_poblacion[anio_maximo].iloc[0]
factor_urgencias_15_a_18 = 1.3

# Extrae los factores de categorizacion para proyectar
factores_categorizacion = porcentaje_categorizacion_rem[anio_maximo].to_frame()
factores_categorizacion["criterio_eleccion"] = "Maximo de urgencias por persona histórico SSMN"
factores_categorizacion = factores_categorizacion.rename(columns={anio_maximo: "factor"})

# Consolida los factores de urgencias
resumen_factores_elegidos = pd.DataFrame(
    index=["urgencias_por_pcte_0_a_14", "urgencias_por_pcte_15_a_18"],
    columns=["factor", "criterio_eleccion"],
    data=[
        [factor_urgencias_0_a_14, "Maximo de urgencias por persona histórico SSMN"],
        [factor_urgencias_15_a_18, "Criterio Experto - Dr Tapia"],
    ],
)

# Concatena factores de categorizacion a las variables
resumen_factores_elegidos = pd.concat([resumen_factores_elegidos, factores_categorizacion])

In [247]:
# Define cuantas consultas habran por grupo etario
consultas_0_a_15 = poblacion_fonasa_extrapolada_ssmn_0_a_14 * factor_urgencias_0_a_14
consultas_15_a_18 = poblacion_fonasa_extrapolada_ssmn_15_a_18 * factor_urgencias_15_a_18

# Define el total de consultas
total_consultas = (consultas_0_a_15 + consultas_15_a_18).to_frame(
    name="Total Consultas Urgencia Proyectadas"
)

print(f"Total consultas urgencia proyectadas 2035: {total_consultas.loc['2035'].iloc[0]:.2f}")

Total consultas urgencia proyectadas 2035: 165838.04


In [258]:
consultas_c1 = (
    total_consultas["Total Consultas Urgencia Proyectadas"]
    * factores_categorizacion.loc["C1", "factor"]
)
consultas_c2 = (
    total_consultas["Total Consultas Urgencia Proyectadas"]
    * factores_categorizacion.loc["C2", "factor"]
)
consultas_c3 = (
    total_consultas["Total Consultas Urgencia Proyectadas"]
    * factores_categorizacion.loc["C3", "factor"]
)
consultas_c4 = (
    total_consultas["Total Consultas Urgencia Proyectadas"]
    * factores_categorizacion.loc["C4", "factor"]
)
consultas_sc = (
    total_consultas["Total Consultas Urgencia Proyectadas"]
    * factores_categorizacion.loc["SIN CATEGORIZACION", "factor"]
)
consultas_c5 = (
    total_consultas["Total Consultas Urgencia Proyectadas"]
    * factores_categorizacion.loc["C5", "factor"]
)

total_categorizacion = pd.DataFrame(
    {
        "C1": consultas_c1,
        "C2": consultas_c2,
        "C3": consultas_c3,
        "C4": consultas_c4,
        "SIN CATEGORIZACION": consultas_sc,
        "C5": consultas_c5,
    }
)

## Resumen MINSAL

In [248]:
datos_a_guardar = {
    "Proyeccion Urgencias RDR": {
        "poblaciones_FONASA_2018_a_2022": poblaciones_fonasa_ssmn.reset_index(),
        "consultas_REM_urgencia": produccion_urgencias_rem.reset_index(),
        "factores_urgencias_por_pcte_0_a_14": proporcion_urgencias_y_poblacion.reset_index(),
        "factores_urgencias_elegidos": resumen_factores_elegidos.reset_index(),
        "poblaciones_FONASA_extrapoladas": poblacion_fonasa_extrapolada_ssmn_0_a_18.reset_index(),
        "consultas_urgencia_proyectadas": total_consultas.reset_index(),
    },
}

In [249]:
ruta_archivo = "../data/interim/5.0_estimacion_consultas_urgencia.xlsx"
guardar_dict_en_excel(ruta_archivo, datos_a_guardar, espacio_filas=3)